# ModelingToolkit.jl

- [Simulating Big Models in Julia with ModelingToolkit @ JuliaCon 2021 Workshop](https://youtu.be/HEVOgSLBzWA)
- [ModelingToolkit.jl](https://github.com/SciML/ModelingToolkit.jl): Symbolic representations for modeling numerical systems.

In [ ]:
using ModelingToolkit
using DifferentialEquations
using Plots
using LinearAlgebra

In [ ]:
# Independent (time) and dependent variables (x nad RHS)
@variables t x(t) RHS(t)  
# parameters = constants throughout the simulation
@parameters τ
# Differential operator w.r.t. time
D = Differential(t)

# Equations in MTK use the tilde character (~) as equality.
# Every MTK system requires a name. The @named macro simply ensures that the symbolic name matches the name in the REPL.
@named fol_separate = ODESystem([ RHS  ~ (1 - x)/τ,  D(x) ~ RHS ])

`structural_simplify()` transforms simple DAEs with dependent terms to ODEs and reduces the number of state variables.

As a plus, the eleminated terms are still trackable by MTK so that you can plot them later on.

In [ ]:
sol = let 
    model = structural_simplify(fol_separate)
    u0 = [x => 0.0]
    tspan = (0.0,10.0)
    param = [τ => 3.0]
    prob = ODEProblem(model, u0, tspan, param)
    solve(prob)
end

# The eliminated term is still tracible
plot(sol, vars=[x, RHS], legend=:right)

## Time-variant external force

If the function is too complex and/or has discontinuity, one should apply `@register_symbolic` to the function to exclude it from symbolic transformations and use it as-is.

In [ ]:
@variables t x(t) f(t)
@parameters τ
D = Differential(t)

value_vector = randn(10)

# Define a time-dependent random external force
f_fun(t) = t >= 10 ? value_vector[end] : value_vector[Int(floor(t))+1]

# "Register" arbitrary Julia functions to be excluded from symbolic transformations but just used as-is.
@register_symbolic f_fun(t)

@named fol_external_f = ODESystem([f ~ f_fun(t), D(x) ~ (f - x)/τ])

In [ ]:
prob = ODEProblem(structural_simplify(fol_external_f), [x => 0.0], (0.0,10.0), [τ => 0.75])
sol = solve(prob)
plot(sol, vars=[x,f])

## Second order ODE system

`ode_order_lowering(eqs)` can automatically transform a second-order ODE into two first-order ODEs.

In [ ]:
# Non-state variables
@parameters σ ρ β
# state variable with time dependence. time (t) is an independent variable
@variables t x(t) y(t) z(t)

# Differential operator
D = Differential(t)

eqs = [D(D(x)) ~ σ * (y-x),
       D(y) ~ x * (ρ - z) - y,
       D(z) ~ x * y - β * z]

In [ ]:
@named sys = ODESystem(eqs)
sys = ode_order_lowering(sys)

In [ ]:
u0 = [ D(x) => 2.0,
        x => 1.0,
        y => 0.0,
        z => 0.0]

p = [σ => 28.0,
     ρ => 10.0,
     β => 8/3]

tspan = (0.0, 100.0)
prob = ODEProblem(sys, u0, tspan, p, jac=true)
sol = solve(prob)

using Plots
plot(solve(prob), vars=(x, y, z), lab="Trajectory")

## Composing systems

By defining connection equation(s) to couple ODE systems together, we can build component-based, hierarchical models.

In [ ]:
@parameters σ ρ β
@variables t x(t) y(t) z(t)

D = Differential(t)

# Lorenz system Eqs
eqs = [D(x) ~ σ * (y - x),
       D(y) ~ x * (ρ - z) - y,
       D(z) ~ x * y - β * z]

@named lorenz1 = ODESystem(eqs)
@named lorenz2 = ODESystem(eqs)

In [ ]:
# Define linking relations
@variables a(t)
@parameters γ

connections = [0 ~ lorenz1.x + lorenz2.y + a * γ]

In [ ]:
@named connLorenz = ODESystem(connections, #=independent var=# t ,  [a], [γ], #=lower sys=# systems = [lorenz1, lorenz2])

In [ ]:
states(connLorenz)

In [ ]:
u0 = [lorenz1.x => 1.0, lorenz1.y => 0.0, lorenz1.z => 0.0,
      lorenz2.x => 0.0, lorenz2.y => 1.0, lorenz2.z => 0.0,
      a => 2.0]

p = [lorenz1.σ => 10.0, lorenz1.ρ => 28.0, lorenz1.β => 8/3,
     lorenz2.σ => 10.0, lorenz2.ρ => 28.0, lorenz2.β => 8/3,
     γ => 2.0]

tspan = (0.0, 100.0)

using Plots

plot(solve(ODEProblem(connLorenz, u0, tspan, p), Rodas5()), vars=(a, lorenz1.x, lorenz2.x))

## Convert existing functions into MTK ones

`modelingtoolkitize(prob)`

And it can generate analytic Jacobin function for faster solving.

Example: **[DAE index reduction](https://mtk.sciml.ai/stable/mtkitize_tutorials/modelingtoolkitize_index_reduction/)** for the pendulum problem, which cannot be solved by regular ODE solvers.

In [ ]:
function pendulum!(du, u, p, t)
    x, dx, y, dy, T = u
    g, L = p
    du[1] = dx
    du[2] = T*x
    du[3] = dy
    du[4] = T*y - g
    # Do not write your function like this after you've learned MTK
    du[5] = x^2 + y^2 - L^2
    return nothing
end

# mass matrix: The last term is constrained
pendulum_fun! = ODEFunction(pendulum!, mass_matrix = Diagonal([1, 1, 1, 1, 0]))

In [ ]:
u0 = [1.0, 0.0, 0.0, 0.0, 0.0]
p = [9.8, 1.0]
tspan = (0.0, 10.0)

In [ ]:
prob0 = ODEProblem(pendulum_fun!, u0, tspan, p)
@named tracedSys = modelingtoolkitize(prob0)

In [ ]:
# Actually, structural_simplify() could do dae_index_lowering() without specifying
# Transform the index-3 DAE into an index-0 ODE 
pendulumSys = structural_simplify(dae_index_lowering(tracedSys))

In [ ]:
# u0 is included in the system already so we don't have to provide it.
prob = ODAEProblem(pendulumSys, [], tspan)
sol = solve(prob, abstol=1e-8, reltol=1e-8)
plot(sol, vars=states(tracedSys))

# Beyond ODEs

## Solving non-linear systems

In [ ]:
using ModelingToolkit, NonlinearSolve
@variables x y z
@parameters σ ρ β

In [ ]:
eqs = [ 0 ~ σ * (y-x),
        0 ~ x * (ρ - z) - y,
        0 ~ x * y - β * z]

In [ ]:
@named ns = NonlinearSystem(eqs, [x, y, z], [σ, ρ, β])

In [ ]:
guess = [x => 1.0, y => 0.0, z => 0.0]
ps = [σ => 10.0, ρ => 26.0, β => 8/3]
prob = NonlinearProblem(ns, guess, ps)
# Should be all zeroes
sol = solve(prob, NewtonRaphson())

In [ ]:
@parameters t
@variables u1(t) u2(t) u3(t) u4(t) u5(t)

eqs = [
    0 ~ u1 - sin(u5)
    0 ~ u2 - cos(u1)
    0 ~ u3 - hypot(u1, u2)
    0 ~ u4 - hypot(u2, u3)
    0 ~ u5 - hypot(u4, u1)
]

@named sys = NonlinearSystem(eqs, [u1, u2, u3, u4, u5], [])

In [ ]:
simple_sys = structural_simplify(sys)

In [ ]:
prob = NonlinearProblem(simple_sys, [u5=>0.0])
sol = solve(prob, NewtonRaphson())

@show sol[u5] sol[u1];  # 1.6 and 1.0

## Stochastic Differential Equations (SDEs)

`SDESystem(equations, noises, iv, dv, ps)`

In [ ]:
using ModelingToolkit, DifferentialEquations, Plots

In [ ]:
@parameters σ ρ β
@variables t x(t) y(t) z(t)

D = Differential(t)

eqs = [D(x) ~ σ * (y - x),
       D(y) ~ x * (ρ - z) - y,
       D(z) ~ x * y - β * z]

# diagonal noise, 10%
noises = [0.1x, 0.1y, 0.1z]

@named de = SDESystem(eqs, noises, t, [x, y, z], [σ,ρ,β])

In [ ]:
u0 = [x => 10.0, y => 10.0, z=> 10.0]
p = [σ => 10.0, ρ => 28.0, β => 8/3]
tspan = (0.0, 200.0)
prob = SDEProblem(de, u0, tspan, p)
sol = solve(prob)
plot(sol, vars=(x, y, z))